## OCR = Text detection + Text recognition

![image](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-00.max-800x600.png)
- 문자의 영역을 검출 = Text detection
- 검출된 영역의 문자를 인식 = Text recognition


## 학습 내용
2. Before Deep Learning
    - 딥러닝 방식이 적용되기 이전의 글자 인식 방법과 Tesseract OCR을 통한 처리과정에 대해 알아보자.

3. Text Detection
    - 딥러닝을 이용해 이미지에서 텍스트의 위치를 찾는 Text Detection 기법에 대해 알아보자.

4. Text Recognition
    - Unsegmented Data가 가지는 특징에 대해 살펴보고, 이를 위한 CRNN 모델에 대해 배워보자.

5. Text recognition + Attention
    - Attention과 Transformer가 OCR 분양에 가져온 변화에 대해 알아보자.

## 16-2. Before Deep Learning
- OCR(Optical Character Recognition)
- 딥러닝 이전의 OCR도 Text detection + Text recognition이라는 기본 흐름에는 큰 차이가 없다.

## 16-3. Text detection
- 이미지에서 텍스트의 위치를 찾는 방법

=> Object detection이나 Segmentation을 위한 기법들
<br/><br/>
- 하지만 이미지 속에서 물체를 찾아낼 때 문자는 조금 다른 특성을 가지고 있다.
- 문자는 몇 개가 모여서 단어 혹은 문장을 이루고 있어서 이미지 내에서 문자를 검출해낼 때엔 검출하기 위한 최소 단위를 정해야 한다.
<br/><br/>
- 문장 단위로 위치를 찾을 경우,
    - 엄청나게 긴 문장과 짧은 길이를 모두 효과적으로 찾아내야 하지만,
- 글자 단위로 위치를 찾을 경우,
    - 글자를 놓치지 않을 수 있지만 맥락에 맞게 묶어주는 추가 과정이 필요하다.
<br/><br/>

![image](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-02.max-800x600.png)
- 위는 다양한 Text detection 기법을 정리한 것이다.
- Text의 바운딩 박스를 구하는 방식이 주류였다.
- 가로 방향(Horizontal)으로만 텍스트 박스를 구하는 방식
- 기울어지거나 세로 방향 등의 여러 방향(Multi-oriented)의 텍스트 박스를 구하는 방식 등
- (e)의 경우 전체 파이프라인의 길이가 짧고 간결해서 빠르면서도 정확한 Text detection 성능을 보인다.
<br/><br/>

- 위 그림에서 단어 단위의 탐지와 글자 단위의 탐지가 모두 활용되고 있다.
1. 단어 단위의 탐지(Word-level Detection)
    - Object detection의 Regression 기반의 Detection 방법
    - Bounding box regression을 통해 각 Anchor의 위치와 크기를 조정하여 정확한 단어의 Bounding box를 얻는다.
> Object detection의 Regression 기반의 Detection 방법은 물체의 위치와 크기를 예측하는 모델
> - 이미지 내에서 물체가 있을 것으로 예상되는 위치에 여러 개의 Anchor를 정의
> - Anchor를 통해 물체의 Bounding box를 예측하는 방식

2. 글자 단위의 탐지(Character-level Detection)
    - Bounding box regression을 사용하여 대신 글자 영역을 Segmentation하는 방법
    - 이미지를 글자 단위로 나누는 Segmentation 모델을 활용하여 각 글자의 위치를 파악
    - 텍스트 영역을 픽셀 수준에서 정확하게 분할하므로 글자의 윤곽을 정확하게 추출할 수 있다.

3. 또한 두 가지 방법을 모두 활용한 방법이 있다.

### 1. Regression
![image](https://d3s0tskafalll9.cloudfront.net/media/images/architecture_of_textboxes.max-800x600.png)
- TextBoxes 이전에는 글자 단위로 인식한 후 결합하는 방식을 취해 왔지만,
- 위 논문에서는 딥러닝 기반의 Detection을 이용하여 단어 단위로 인식한다.
<br/><br/>
- 일반적으로 단어들은 가로로 길기 때문에 Aspect ratio(종횡비)가 크다.
- 이에 따라 몇 가지 변형을 주게 된다.
- 기존의 SSD에서는 Regression을 위한 Convolution layer에서 3x3 크기의 kernel을 갖는다.
<br/><br/>
- 하지만 여기서는 긴 단어의 feature를 활용하기 위해서 1x5로 convolution filter를 정의하여 사용한다.
- Anchor box의 aspect ratio를 1, 2, 3, 5, 7로 만들고,
- 이에 vertical offset을 적용하여 세로 방향으로 촘촘한 단어의 배열에 대응하도록 한다.
<br/><br/>

> 종횡비(Aspcet ratio)
> - 종횡비는 가로와 세로 길이의 비를 의미한다.
> - 가로세로비, 영상비, 화면비 등으로 부르기도 한다.
> - 일반적으로 16:9와 같이 '가로:세로'의 형태로 표현한다.
> - 그러나 논문에서는 가로와 세로의 비를 계산하여 하나의 숫자로 표현하고 있다.
<br/><br/>

> offset
> - 일반적인 의미의 Offset이란 특정한 값에서 차이가 나는 값 또는 차이를 의미한다.
> - 주의할 점은 offset과 오차가 다르다는 것이다.
> - offset은 차이가 목적에 의해 만들어진 것과 상황에 따라 자연스럽게 발생된 것을 모두 포함하기 때문이다.

### 2. Segmentation
![image](https://d3s0tskafalll9.cloudfront.net/media/images/segmentation_map.max-800x600.png)
- Semantic segmentation을 이미지 내의 영역을 class로 분리해내는 방법
- 이 방법을 문자의 영역을 찾기 위해서 적용하면, 배경과 글자인 영역으로 분리할 수 있다.
- 하지만 문자들은 매우 촘촘히 배열되어있으니 글자 영역으로 찾아낸 뒤에 이를 분리해내는 작업이나 연결하는 작업을 더 해서 원하는 최소단위로 만들어줘야 한다.
<br/><br/>
- PixelLink는 Text 영역을 찾아내는 segmentation과 함께,
- 글자가 어느 방향으로 연결되는지를 같이 학습하여 Text 영역 간의 분리 및 연결을 할 수 있는 정보를 추가적으로 활용하고 있다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/architecture_pixellink.max-800x600.png)
- PixelLink의 전체적인 구조는 U-Net과 유사하다.
- 하지만 ouput으로 총 9가지의 정보를 얻는다.
<br/><br/>
- 위 그림의 녹색 부분이 input과 output을 의미한다.
- output 중 하나는 Text/non-test Prediction을 위한 class segmentation map으로
    - 해당 영역이 Text인지 Non-Text인지 예측값을 의미하는 2개의 커널을 가진다.
- 나머지 8가지
    - 글자의 Pixel을 중심으로 인접한 8개의 Pixel에 대한 연결 여부를 의미하는 16개의 커널로 이루어진 Link Prediction map이다.
<br/><br/>

- 위 그림을 보면 conv 1x1, 2(16) 형태의 레이어가 U-Net 구조와 연결되어, 인접 pixel간 연결 구조가 지속적으로 유지되도록 하는 모델 구조임을 알 수 있다.
- 이를 통해서 인접한 pixel이 중심 pixel과 단어 단위로 연결된 pixel인지,
- 아니면 분리된 pixel인지 알 수 있으므로,
- 문자 영역이 단어 단위로 분리된 Instance segmentation이 가능해진다.

### 3. 최근의 방법들
- 위에서 확인한 TextBoxes나 PixelLink는 3년 전에 공개된 논문들이다.
- 최근에는 어떤 방식으로 문자 영역을 찾아내는지 알아보자.

#### 3.1. CRAFT
![image](https://d3s0tskafalll9.cloudfront.net/media/images/craft_affinity_map.max-800x600.png)
- CRAFT는 문자 단위로 문자의 위치를 찾아낸 뒤,
- 이를 연결하는 방식을 Segmentation 기반으로 구현한 방법이다.
- 문자의 영역을 boundary로 명확히 구분하지 않고,
- 가우시안 분포를 따르는 원형의 score map을 만들어서 배치시키는 방법으로 문자의 영역을 학습한다.
<br/><br/>
- 문자 단위 라벵르 가진 데이터셋이 많지 않기 때문에,
- 단어 단위의 정보만 있는 데이터셋에 대해 단어의 영역에 Inference를 한 후,
- 얻어진 문자 단위의 위치를 다시 학습에 활용하는 Weakly supervised leraning을 활용했다.

#### 3.2. Pyramid Mask Text Detector
![image](https://d3s0tskafalll9.cloudfront.net/media/images/PMTD.max-800x600.png)
- PMTD(Pyramid Mask Text Detector)는 Mask-RCNN의 구조를 활용하였다.
1. 먼저 Text 영역을 Region proposal network로 찾아낸다.
2. 그다음 Box head에서 더 정확하게 regression 및 classification을 하고 Mask head에서 Instance의 Segmentation을 하는 과정을 거친다.
<br/><br/>
- PMTD는 Mask 정보가 부정확한 경우를 반영하기 위해서 Soft-segmentation을 활용한다.
- 이전의 Mask-RCNN의 경우,
    - 단어 영역이 Box head에 의해 빨간색으로 잡히면 위의 이미지의 baseline처럼 boundary 모두 Text 영역으로 잡지만,
- PMTD는 단어의 사각형 배치 특성을 반영하여 피라미드 형태의 Score map을 활용한다.
- 따라서 Pyramid 형상의 Mask를 갖게 되어 Pyramid Mask Text Detector라는 이름을 붙였다.

## 16-4. Text recognition
### 1. Unsegmented Data
![image](https://d3s0tskafalll9.cloudfront.net/media/images/BEN-RO1.max-800x600.jpg)
- 글자 이미지들은 문자의 순서대로 정보를 가지고 있다.
- 위의 "YOU"라는 이미지에서 "Y", "O", "U"의 영역은 이미지상에서 분리가 가능하다.
<br/><br/>
- 하지만 이와 다르게 분리에 드는 비용이 많이 들거나 어려워 Segmenation이 되어있지 않은 데이터를 Unsegmented data라고 한다.
- 이미지에서 문자를 읽어내는 OCR 분야뿐만 아니라 다른 분야들에서도 이런 데이터를 볼 수 있다.
<br/><br/>
- 예를 들어,
- 아래 그림과 같이 annotation이 제대로 안 된 음성데이터도 Unsegmented data의 한 종류이다.
- Waveform 파일에 대한 라벨이 "the sound of"라고 달려 있다고 할 때,
- 해당 waveform 파일에서 어디까지가 각각 "the", "sound", "of"인지의 경계가 명확하게 분할되어 있지 않은 경우를 생할 수 있다.
![image](https://d3s0tskafalll9.cloudfront.net/media/images/unsegmented_data_copy.max-800x600.png)

```
정리
- Unsegmented Data(세분화되지 않은 데이터)
- 주요 특징 : 이 데이터가 하위 구성 요소들 간에 분리되어 있지 않고, 연속적인 시퀀스를 형성하고 있다는 것
- 간단히 말하면
- 데이터가 세분화되어 있지 않다는 것은
=> 데이터의 부분들이 명확한 경계나 구분 없이 이어져 있는 상태
=> 이 부분들이 서로 구분되지 않고 연결되어 있으며, 전체 데이터가 하나의 연속된 시퀀스를 형성한다.

ex) 텍스트 데이터에서 문장이나 단어들이 명확한 경계나 구분 없이 이어져 있다면,
- 그 데이터는 세분화되지 않은(unsegmented) 상태이다.
- 이 경우에는 어떤 단어가 어디에서 시작하고 끝나는지 구별하기 어렵다.
```

### 2. CNN과 RNN의 만남 CRNN
![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/crnn.png)
![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/crnn_structure.png)
![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/image_oZbhrom.png)
![image](https://vitalab.github.io/article/images/lenet/a384.gif)
- Unsegmented Data들이 가진 주요한 특징 중 하나는 segment되어 있지 않은 하위 데이터들끼리 시퀀스를 이루고 있다는 점
- 그래서 연구자들이 생각해 낸 방법 중 하나는 CNN과 RNN을 같이 쓰는 방법이다. 이런 모델을 CRNN이라고 한다.
<br/><br/>
- CNN
    - 문자 이미지에서 정보를 추출하기 위해서는 Feature Extractor가 필요하다.
    - Feature Extractor로 사용되는 CNN 기반의 VGG 또는 ResNet과 같은 네트워크로부터 문자의 정보를 가진 Feature를 얻어낼 수 있다.
- RNN
    - 이렇게 추출된 Feature를 Map-To-Sequence를 통해 Sequence 형태의 feature로 변환한 후,
    - 다양한 길의 Input을 처리할 수 있는 RNN으로 넣는다.
<br/><br/>

- RNN이 Feature로부터 문자를 인식하기 위해서는 문자 영역처럼 넓은 정보가 필요하기 때문에 LSTM으로 구성한다.
- 앞의 정보뿐만 아니라 뒤의 정보가 필요하기 때문에 이를 Bidirectional로 구성해서 Bidirectional LSTM을 사용했다.
- Bidirectional LSTM을 사용해 step마다 나오는 결과는 Transcription Layer에서 문자로 변환된다.

```
정리

Feature Extractor(특징 추출기)
- 문자 이미지로부터 중요한 특징들을 추출하는 역할을 한다.
- CNN 기반의 아키텍처인 VGG 또는 ResNet과 같은 네트워크를 사용한다.
- 이 네트워크는 이지미에서 에지, 색상, 질감 등과 같은 시각적 틍징을 학습하여 추출한다.

Map-To-Sequence(맵-투-시퀀스)
- CNN을 통해 얻어진 고차원의 특징 맵을 시퀀스 형태로 변환하는 단계이다.
- 일반적으로 이 특징 맵을 1D로 펼치거나,
- 다양한 방식으로 변환하여 시퀀스로 만든다.
- 이는 추출된 고수준 특징을 순차적인 정보로 변환하는 과정이다.
```

### 3. CTC
- CRNN에서는 Step마다 Fully Connected Layer의 logit을 Softmax 함수에 넣어줌으로써 어떤 문자일 확률이 높은지 알 수 있다.
- 하지만 이 결과 그대로 문자로 변환하면 기대한 것과 다른 결과가 나온다.
- 왜냐하면 모델의 Output은 24개의 글자로 이루어진 Sequence이지만, 실제 결과는 이와 다르기 때문이다.
<br/><br/>
- 예를 들어 "HELLO"라는 이미지가 들어오면
- Output이 "HHHEEELLLOOOOO..."와 같이 24자의 sequence를 보게 된다.
- 그렇다면 이 24자의 Sequence를 실제 인식 결과로 바꾸기 위해서는 어떤 방법을 써야할까?

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/ctc.png)
- CRNN에서는 Unsegmented Data를 위해 CTC(Connectionist Temporal Classification)를 활용한다.
- CTC는 Unsegmented data와 같이 Input과 Ouput이 서로 다른 Length의 Sequence를 가질 때, 이를 Align 없이 활용하는 방법이다.
- CTC의 핵심인 모델의 Output에서 Label sequence의 확률을 구할 수 있는 방법에 대해 알아보자.
<br/><br/>
- "HELLO"로 만들기 위해서는 중복되는 단어를 제거해서 "HELO"가 될 것이다.
- 그렇다면 "L"이 두 번 중복되는 경우에는 Label Encode에서 이렇게 같은 문자를 구분하기 위한 Blank를 중된 라벨 사이를 구분하기 위해 넣어준다.
- Blank token을 '-'로 대신하여 Output을 만드는 Decoder를 의미한다.
- Decode 후에 중복을 제거하고, 인식할 문자가 아닌 값을 지워주면 "HELLO"라는 결과를 얻을 수 있다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/wbHRk.png)

### 4. TPS
- 책이 아닌 거리의 글자를 읽을 때, OCR이 어려워지는 이유는 불규칙한 방향이나 휘어진 진행 방향 때문이다.
- TPS(Thin Plate Spline) Transformation을 적용하여 입력 이미지를 단어 영역에 맞게 변형 시켜 인식이 잘되도록 한다.
![image](https://d3s0tskafalll9.cloudfront.net/media/images/thinplates-dy.max-800x600.png)
- Thin plate spline은 control point를 정의하고 해당 point들이 특정 위치로 옮겨졌을 때,
- 축 방향의 변화를 interpolation하여 모든 위치의 변화를 추정해낸다.
- 이를 통해서 전체 이미지 pixel의 변화를 control point로 만들어낼 수 있다.
<br/><br/>

![image](https://d3s0tskafalll9.cloudfront.net/media/images/GC-6-L-03.max-800x600.png)
- 논문에서는 Control point 20개를 미리 정의한다.
- Spatial Transformer Network를 통해서 Control point가 얼마나 움직여야 하는지 예측하는 네트워크를 Recognition model 앞단에 붙여 입력 이미지를 정방향으로 맞춘다.
- TPS 연산은 미분 가능한 연산이기 때문에 이 모듈을 Recognition model 앞단에 붙여서 학습이 바로 가능하다.

> Spatial Transformer Network
> - input 이미지에 크기, 위치, 회전 등의 변환을 가해 추론을 더욱 용이하게 하는 transform matrix를 찾아 매핑해 주는 네트워크이다.


## 16-5. Text recognition + Attention
### 1. Attention sequence prediction
![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/rnn_decoder.png)
- CTC를 활용한 CRNN의 경우,
- column에 따라서 prediction된 Label의 중복된 것들을 제거해 줌으로써 원하는 형태의 label로 만들어줬다.
> 쉽게 말해
> CTC는 column에 따라 예측한 Label의 중복을 제거하여 원하는 형태로 만들었다.
<br/><br/>
- Attention 기반의 sequence prediction은 문장의 길이를 고정하고,
- 입력되는 feature에 대한 Attention을 기반으로 해당 글자의 Label을 prediction 한다.
- RNN으로 Character label을 뽑아낸다고 생각하면 된다.
- 첫번째 글자에서 입력 feature에 대한 Attention을 기반으로 label을 추정하고,
- 추정된 label을 다시 입력으로 사용하여 다음 글자를 추정해내는 방식이다.
> Attention
> - 입력 데이터의 특정 부분에 모델이 집중하도록 하는 기술
> - 입력 시퀀스의 중요한 부분에 집중하며, 출력을 생성하거나 다음 단계의 예측을 수행할 때 해당 부분을 고려한다.
<br/><br/>
- 20글자를 뽑겠다고 정하게 되면 "YOU"같은 경우에는 3글자를 채우고 빈자리가 문제가 된다.
- 이러한 경우를 위해 미리 정해둔 Token을 사용한다.
- 이 Token에는 처음에 사용되는 "start" Token과 끝에 사용되는 "end" Token이 있다.
- 필요에 따라서 예외처리나 공백을 위한 token을 만들어서 사용하기도 한다.

### 2. Transformer와 함께!
![image](https://d3s0tskafalll9.cloudfront.net/media/images/transformer_rec.max-800x600.png)
- Transformer도 Recognition 모델에 활용되기 시작한다.
- Transformer는 Query, Key, Value라는 개념을 통해서 Self-Attention을 입력으로부터 만들어낸다.
- 이를 통해서 입력에서 중요한 Feature에 대해 Weight를 주게 된다.
- Attention의 핵심은 Decoder의 현재 포지션에서 중요한 Encoder의 State에 가중치가 높게 매겨진다는 점이다.